In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets
import torchmetrics
from torchmetrics import ConfusionMatrix
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [2]:
# resizes and transforms images

transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),])

# applies transformations and loads datasets

test_dataset = datasets.ImageFolder('M156_Dataset/test', transform = transform)
train_dataset = datasets.ImageFolder('M156_Dataset/train', transform = transform)

In [3]:
# loads data in dataloaders

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=False)

# Replace the last fully connected layer 
num_ftrs = model.fc.in_features

# Modify the fully connected layers
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 4)  # Output layer with 4 units
)

model = model.to(device)

C:\Users\Ben\AppData\Local\anaconda3\envs\NLPTransformer\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Ben\AppData\Local\anaconda3\envs\NLPTransformer\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
model.load_state_dict(torch.load('model_weights.pth'))
model.eval() #set the model to evaluation mode
criterion = nn.CrossEntropyLoss().to(device)

C:\Users\Ben\AppData\Local\Temp\ipykernel_18268\3770576977.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_weights.pth'))


In [6]:
# Initialize confusion matrix
conf_mat = ConfusionMatrix(task="multiclass", num_classes=4).to(device)
# Initialize a list to store true labels and predictions for metrics calculation
all_preds = []
all_labels = []

# Validation phase
model.eval()
validation_loss = 0.0

with torch.no_grad():
    for feature, target in train_loader:
        target = target.to(device)
        feature = feature.to(device)

        # Forward pass
        prediction = model(feature)

        # Store predictions and labels
        all_preds.extend(prediction.argmax(dim=1).cpu().numpy())
        all_labels.extend(target.cpu().numpy())

        # Calculate loss
        loss = criterion(prediction, nn.functional.one_hot(target, num_classes=4).double().to(device))
        validation_loss += loss.item()

# Compute the confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

# Calculate F1 Score, Precision, and Recall
f1 = f1_score(all_labels, all_preds, average='weighted')
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')

print("F1 Score: ", f1)
print("Precision: ", precision)
print("Recall: ", recall)

# Classification report (optional)
from sklearn.metrics import classification_report
print("Classification Report:\n", classification_report(all_labels, all_preds))

Confusion Matrix:
 [[353  38  12   0]
 [  5 799   0   0]
 [  0   1 774   0]
 [  0   1   0 642]]
F1 Score:  0.9778661223661544
Precision:  0.9787459430360792
Recall:  0.9782857142857143
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.88      0.93       403
           1       0.95      0.99      0.97       804
           2       0.98      1.00      0.99       775
           3       1.00      1.00      1.00       643

    accuracy                           0.98      2625
   macro avg       0.98      0.97      0.97      2625
weighted avg       0.98      0.98      0.98      2625



In [7]:
# Initialize confusion matrix
conf_mat = ConfusionMatrix(task="multiclass", num_classes=4).to(device)

# Initialize a list to store true labels and predictions for metrics calculation
all_preds = []
all_labels = []

# Validation phase
model.eval()
validation_loss = 0.0

with torch.no_grad():
    for feature, target in test_loader:
        target = target.to(device)
        feature = feature.to(device)

        # Forward pass
        prediction = model(feature)

        # Store predictions and labels
        all_preds.extend(prediction.argmax(dim=1).cpu().numpy())
        all_labels.extend(target.cpu().numpy())

        # Calculate loss
        loss = criterion(prediction, nn.functional.one_hot(target, num_classes=4).double().to(device))
        validation_loss += loss.item()

# Compute the confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

# Calculate F1 Score, Precision, and Recall
f1 = f1_score(all_labels, all_preds, average='weighted')
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')

print("F1 Score: ", f1)
print("Precision: ", precision)
print("Recall: ", recall)

# Classification report (optional)
from sklearn.metrics import classification_report
print("Classification Report:\n", classification_report(all_labels, all_preds))

Confusion Matrix:
 [[42  6  2  0]
 [ 1 99  0  0]
 [ 0  0 97  0]
 [ 0  1  0 79]]
F1 Score:  0.9687610232553339
Precision:  0.9702563906967093
Recall:  0.9694189602446484
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.84      0.90        50
           1       0.93      0.99      0.96       100
           2       0.98      1.00      0.99        97
           3       1.00      0.99      0.99        80

    accuracy                           0.97       327
   macro avg       0.97      0.95      0.96       327
weighted avg       0.97      0.97      0.97       327

